In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df=pd.read_csv('fordTrain.csv')
df1=pd.read_csv('fordTest.csv')
df=df.drop(['TrialID','ObsNum','P8','V7','V9'],axis=1) 
df1=df1.drop(['TrialID','ObsNum','P8','V7','V9'],axis=1)

In [3]:
df=pd.read_csv('fordTrain.csv')
df1=pd.read_csv('fordTest.csv')
df=df.drop(['TrialID','ObsNum','P8','V7','V9'],axis=1) 
df1=df1.drop(['TrialID','ObsNum','P8','V7','V9'],axis=1)
y_train=df['IsAlert']  # The labels for the train set
#X_train=df.drop(['IsAlert'],axis=1) # The features of the train set

y_test=df1['IsAlert'] # The labels for the test set
#X_test=df1.drop(['IsAlert'],axis=1) # The features of the test set
X_train=df.drop(['IsAlert','P1','P2','P3','P4','P5','E1','E2','E3','E3','E4','E5','E6','E9','E11','V1','V2','V3','V4','V5','V6','V8','V10'],axis=1)
X_test=df1.drop(['IsAlert','P1','P2','P3','P4','P5','E1','E2','E3','E3','E4','E5','E6','E9','E11','V1','V2','V3','V4','V5','V6','V8','V10'],axis=1)

In [5]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
#Gradient Boosting
model1 = GradientBoostingClassifier(criterion='friedman_mse',
                                    learning_rate= 0.5,
                                    loss= 'deviance', 
                                    max_depth= 8, 
                                    n_estimators= 100)

#model1 = model1.fit(X_train,y_train)

#XGBoost
model2 = XGBClassifier(booster='gbtree',
                       gamma= 0, 
                       learning_rate= 0.7,
                       max_depth= 8,
                       n_estimators= 400, 
                       objective= 'binary:logistic')

#model2 = model2.fit(X_train,y_train)

#Random Forest
model3 = RandomForestClassifier(criterion= 'gini',
                                max_depth = 100,
                                min_samples_leaf= 1,
                                min_samples_split= 2,
                                n_estimators= 20)

#model3 = model3.fit(X_train,y_train)

#lightGBM
model4 = LGBMClassifier(boosting_type = 'gbdt',
                        learning_rate = 0.3,
                        max_depth = 7,
                        n_estimators = 400,
                        num_leaves = 31)

In [6]:
def classification_crossvalscore(model, x, y):
    acc = cross_val_score(model, x, y, scoring = 'accuracy', cv = kfold )
    #prec = cross_val_score(model, x, y, scoring = 'precision', cv = 5)
    #recall = cross_val_score(model, x, y, scoring = 'recall', cv = 5)
    #rocauc = cross_val_score(model, x, y, scoring = 'roc_auc', cv = 5)
    return acc#, prec, recall, rocauc
kfold = KFold(n_splits=3, shuffle=False, random_state=42)
for model,label in zip([model1,model2,model3, model4],["GBC","XGB", "RF", "LGBM"]):
              print(label,classification_crossvalscore(model,X_train,y_train).mean())

('GBC', 0.955660355057561)
('XGB', 0.9754036356759568)
('RF', 0.9827151738647378)
('LGBM', 0.9460463796893838)


In [7]:
#predict model and stack model
GBC_pred = model1.fit(X_train,y_train).predict(X_test)
XGB_pred = model2.fit(X_train,y_train).predict(X_test)
RF_pred = model3.fit(X_train,y_train).predict(X_test)
LGB_pred = model4.fit(X_train,y_train).predict(X_test)

In [8]:
print("Out Sample scoring-Standalone")


for model,label in zip([model1,model2,model3, model4],["GBC","XGB", "RF", "LGBM"]):
              print(label,classification_crossvalscore(model,X_train,y_train).mean())

Out Sample scoring-Standalone
('GBC', 0.9562891520695932)
('XGB', 0.9754036356759568)
('RF', 0.9826537125026592)
('LGBM', 0.9460463796893838)


In [9]:
from vecstack import stacking

#Initialize first level
models = [model1, model2, model3] #input models as a list for metamodel

#Performing Stacking
S_train, S_test = stacking(models,                     # list of models
                           X_train, y_train, X_test,   # data
                           regression=False,            # regression task (if you need  classification - set to False)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test  //set in each fold and find mean
                           save_dir=None,              # do not save result and log (to save in current dir - set to '.')
                           metric=accuracy_score,      # metric: callable
                           n_folds=3,                  # number of folds
                           shuffle=True,               # shuffle the data
                           random_state=42,             # ensure reproducibility
                           verbose=2)                  # print all info """


# Initialize 2nd level model  with XGB
model = model4
    
# Fit 2nd level model
stack = model.fit(S_train, y_train)

# Predict
#y_pred = model.predict(S_test)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.95664137]
    fold  1:  [0.95666265]
    fold  2:  [0.95590384]
    ----
    MEAN:     [0.95640262] + [0.00035280]
    FULL:     [0.95640262]

model  1:     [XGBClassifier]
    fold  0:  [0.97475356]
    fold  1:  [0.97421459]
    fold  2:  [0.97454790]
    ----
    MEAN:     [0.97450535] + [0.00022208]
    FULL:     [0.97450535]

model  2:     [RandomForestClassifier]
    fold  0:  [0.98226367]
    fold  1:  [0.98227076]
    fold  2:  [0.98263953]
    ----
    MEAN:     [0.98239132] + [0.00017553]
    FULL:     [0.98239132]



In [10]:
print("In Sample scoring-Vecstack")


for model,label in zip([model1,model2,model3, stack],["GBC","XGB", "RF", "Vecstack"]):
              print(label,classification_crossvalscore(model,X_train,y_train).mean())

In Sample scoring-Vecstack
('GBC', 0.9566791953289364)
('XGB', 0.9754036356759568)
('RF', 0.9824220504455949)
('Vecstack', 0.9460463796893838)


In [11]:
#predict model and stack model
GBC_pred = model1.fit(X_train,y_train).predict(X_test)
XGB_pred = model2.fit(X_train,y_train).predict(X_test)
RF_pred = model3.fit(X_train,y_train).predict(X_test)
stack_pred = stack.predict(S_test)  #has to be on the S_test not on X_

In [12]:
#accuracy score out of sample
print("Out of Sample scoring-Vecstack")

for model, label in zip([GBC_pred, XGB_pred, RF_pred, stack_pred],["GBC","XGB", "RF", "Vecstack"]):
    print(label,accuracy_score(y_test, model, normalize = True))

Out of Sample scoring-Vecstack
('GBC', 0.9599722006188671)
('XGB', 0.9808713782205086)
('RF', 0.9883893457768659)
('Vecstack', 0.9872144909789905)


In [13]:
estimators = [('GBC', model1),('XGB', model2),('RF', model3), ('LGBM', model4)]

#Voting Classifier
ecf = VotingClassifier(estimators= estimators,
                          voting='soft',            #soft for probability
                          weights=None,
                          n_jobs=1,
                          flatten_transform=None
                          )


#Voting Classifier fit
vc = ecf.fit(X_train, y_train)

#Voting Classifier predict
VC_pred=vc.predict(X_test)

In [14]:
print("In Sample scoring-Voting Classifier")

for model,label in zip([model1,model2,model3,model4, vc],["GBC","XGB", "RF","LGBM" ,"VC_stack"]):
              print(label,classification_crossvalscore(model,X_train,y_train).mean())

In Sample scoring-Voting Classifier
('GBC', 0.9563837080112522)
('XGB', 0.9754036356759568)
('RF', 0.9825615204595418)
('LGBM', 0.9460463796893838)
('VC_stack', 0.9734108692054937)


In [15]:
#accuracy score out of sample
print("Out of Sample scoring-Voting Classifier")


for model, label in zip([GBC_pred, XGB_pred, RF_pred,LGB_pred,VC_pred],["GBC","XGB", "RF","LGBM" ,"VC_stack"]):
    print(label,accuracy_score(y_test, model, normalize = True))

Out of Sample scoring-Voting Classifier
('GBC', 0.9599722006188671)
('XGB', 0.9808713782205086)
('RF', 0.9883893457768659)
('LGBM', 0.9501321022178831)
('VC_stack', 0.9783727433686893)


In [16]:
from mlxtend.classifier import StackingClassifier

In [17]:
#models classifier
classifiers = [model1,model2,model3]

#stacking classifier
stack_c = StackingClassifier(classifiers = classifiers,
                            meta_classifier=model4)

#mlxtend classifier fit
stack = stack_c.fit(X_train, y_train)

#mlxtend classifier predict
mlx_pred=stack.predict(X_test)

In [18]:
print("In Sample scoring-mlxtend Classifier")

for model,label in zip([model1,model2,model3, vc],["GBC","XGB", "RF", "mlx_stack"]):
              print(label,classification_crossvalscore(model,X_train,y_train).mean())

In Sample scoring-mlxtend Classifier
('GBC', 0.9562655130841785)
('XGB', 0.9754036356759568)
('RF', 0.982511878590171)
('mlx_stack', 0.9735597948136067)


In [19]:
#accuracy score out of sample
print("Out of Sample scoring-mlxtend Classifier")

for model, label in zip([GBC_pred, LGB_pred, RF_pred, mlx_pred],["GBC","XGB", "RF", "mlx_stack"]):
    print(label,accuracy_score(y_test, model, normalize = True))

Out of Sample scoring-mlxtend Classifier
('GBC', 0.9599722006188671)
('XGB', 0.9501321022178831)
('RF', 0.9883893457768659)
('mlx_stack', 0.9884500190293383)
